In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import time
import datetime as dt
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import datetime
from itertools import chain #used for flattening the list of list
import seaborn as sns
from sklearn.model_selection import train_test_split
%matplotlib inline

**part 1**

In [3]:
data_f = pd.read_csv('../input/tmd-datasetcleaned/cleaned.csv')
data_f.head()

**part-a**

In [4]:
print("No_of_Rows: ",data_f.shape[0])
print("No_of_Columns: ", data_f.shape[1])
print("Columns: ",data_f.columns)

**part-b**

In [5]:
users = data_f["user"].unique()
print("No_of_Unique_Users: ", len(users))
print("Users:", users)

 **2**

In [6]:
t_ini = time.time()# t1 is the initial time 



data_f["User_Shift"] = data_f["user"].shift(1)
data_f["Class_Shift"] = data_f["class"].shift(1)
data_f['timestamp'] = pd.to_datetime(data_f['timestamp'])#in case if the time stamp is not uniform
data_f["Time_Shift"] = data_f["timestamp"].shift(1)#shift by 1 so that can be found change 

data_f["is_user_shift"] = (data_f["user"]!=data_f["User_Shift"])
data_f["is_class_shift"] = (data_f["class"]!=data_f["Class_Shift"])

data_f["is_time_shift"] = ((data_f["Time_Shift"]-data_f["timestamp"]).dt.total_seconds() > 10)#check whether the total number of seconds in given duration is > 10 

classes = ["car", "train", "bus", "walk", "bike", "e-bike"]
car, bus, walk, e_bike, bike, train, users_list, i = [], [], [], [], [], [], [], 1

for user in users:
    users_data_f = data_f[data_f["user"]==user].reset_index(drop=True)#by using drop we mean here to make the column as index and removed in the data frame automatically
    users_list.append(user)
    
    d = {'bike': 0, 'car': 0, 'walk': 0, 'bus': 0, 'train': 0, 'e-bike': 0}
    class_shift_points = users_data_f.loc[users_data_f["is_class_shift"]==True]# loc: access a gp of rows and columns by labels or a boolean row
    time_shift_points = users_data_f.loc[users_data_f["is_time_shift"]==True]
    dict1 = dict(class_shift_points["class"].value_counts())#uniquely count the items in that column
    dict2 = dict(time_shift_points["class"].value_counts())
    
    for i in dict1.items():#The items() method returns a view object. The view object contains the key-value pairs of the dictionary, as tuples in a list.
        d[i[0]] = d[i[0]] + i[1]
    for i in dict2.items():
        d[i[0]] = d[i[0]] + i[1]
    
    
    e_bike.append(d['e-bike'])
    bus.append(d['bus'])
    car.append(d['car'])
    train.append(d['train'])
    walk.append(d['walk'])
    bike.append(d['bike'])
    
    
final_table = pd.DataFrame({"user":users_list, "car":car,"bus":bus,  "e-bike":e_bike,"train":train, "bike":bike, "walk":walk})

t_fin = time.time()


In [7]:
print("Total time taken is : ",(t_fin-t_ini), "seconds") 


In [8]:
final_table

**part-3**

In [9]:
t_ini = time.time()#initial time of the window

data_f["is_new_sequence"] = (data_f["is_time_shift"] + data_f["is_class_shift"] + data_f["is_user_shift"])
Seq_num =[]
count = 0

New_Sequences = data_f.loc[data_f["is_new_sequence"]==True]# loc: access a gp of rows and columns by labels or a boolean row
index = New_Sequences.index # making new_sequences a index

for i in range(len(index)-1):
    seq_len = index[i+1]-index[i]
    new_list = [i+1]*seq_len
    Seq_num.append(new_list)
    count = i+1
    
rem_len = len(data_f)-index[count]

Seq_num.append([count+1]*rem_len)

flattened_list = list(chain(*Seq_num))#flattening the list of a list like [1,[1,2]] after this [1,1,2]
sequences = np.unique(flattened_list)# to fetch the unique value in a particular column

data_f["sequence_num"] = flattened_list
win_time=[]
seq_part_num = []

for seq in sequences:
    seq_df = data_f[data_f["sequence_num"]==seq].reset_index(drop=True)
    min_time = seq_df["timestamp"][0]
    min_time_list = [min_time]*len(seq_df)
    min_time_list = pd.to_datetime(min_time_list)#in case if the time stamp is not uniform
    time_diff = (seq_df["timestamp"]-min_time_list).dt.total_seconds()#counting total seconds
    
    window_num = np.floor_divide(time_diff,5)
    window_to_str = window_num.apply(str)#apply() to Convert the Integers to Strings in Pandas DataFrame
    seq_str = [str(seq)]*len(seq_df)
    _str = ["_"]*len(seq_df)
    part_num = list(i+j for i,j in zip(seq_str , _str))#making iterator of tuple through zip
    part_num = list(i+j for i,j in zip(part_num , window_to_str))
    
    win_time.append(window_num)
    seq_part_num.append(part_num)
    
win_time = list(chain.from_iterable(win_time))
seq_part_num = list(chain.from_iterable(seq_part_num))#flattening the list of a list like [1,[1,2]] after this [1,1,2]
data_f["Partition label"] = seq_part_num

df1 = [y for x,y in data_f.groupby(data_f['class'])]#group the data using column 'class'

class_name=[]
windows_final = []
for class_grp in df1:
    class_grp = class_grp.reset_index(drop=True)#by using drop we mean here to make the column as index and removed in the data frame automatically
    Class = class_grp["class"][0]
    class_name.append(Class)
    wind_count = len(class_grp["Partition label"].unique())
    windows_final.append(wind_count)
    
class_wid_table = pd.DataFrame({"Class Name":class_name, "5 second windows in Class":windows_final})

t_fin = time.time()

In [10]:
print("Total time taken for the window partition is: ", (t_fin-t_ini), "seconds")

**********

**4**

In [11]:
labels = data_f["Partition label"].unique()#find the unique values of the column partition label
print("Total 5 second time window: ",len(labels))

In [12]:
label_dfs = [y for x,y in data_f.groupby(data_f['Partition label'])]#groupby using column Partition label 
label = []
label_class = []
features = []

for label_df in label_dfs:
    label_df = label_df.reset_index(drop=True)#by using drop we mean here to make the column as index and removed in the data frame automatically
    
    label.append(label_df["Partition label"][0])
    label_class.append(label_df["class"][0])
    
    x_min = min(label_df["x"])
    y_min = min(label_df["y"])
    z_min = min(label_df["z"])
    
    x_max = max(label_df["x"])
    y_max = max(label_df["y"])
    z_max = max(label_df["z"])
    
    x_avg = np.mean(label_df["x"])
    y_avg = np.mean(label_df["y"])
    z_avg = np.mean(label_df["z"])
    
    x_std = np.std(label_df["x"])
    y_std = np.std(label_df["y"])
    z_std = np.std(label_df["z"])
    
    features.append([x_min, y_min, z_min, x_max, y_max, z_max, x_avg, y_avg, z_avg, x_std, y_std, z_std])
    
feature_columns = ["x_min", "y_min", "z_min", "x_max", "y_max", "z_max", 
                   "x_avg", "y_avg", "z_avg", "x_std", "y_std", "z_std"]
    
features = pd.DataFrame(features, columns = feature_columns)
features["Label"] = label
features["class"] = label_class



In [13]:
features.head()

In [14]:
fig, ax = plt.subplots(nrows=4, ncols=3, figsize=(20,20))

sns.boxplot(x='class', y='x_min', data=features, ax=ax[0][0])
sns.boxplot(x='class', y='y_min', data=features, ax=ax[0][1])
sns.boxplot(x='class', y='z_min', data=features, ax=ax[0][2])
sns.boxplot(x='class', y='x_max', data=features, ax=ax[1][0])
sns.boxplot(x='class', y='y_max', data=features, ax=ax[1][1])
sns.boxplot(x='class', y='z_max', data=features, ax=ax[1][2])
sns.boxplot(x='class', y='x_avg', data=features, ax=ax[2][0])
sns.boxplot(x='class', y='y_avg', data=features, ax=ax[2][1])
sns.boxplot(x='class', y='z_avg', data=features, ax=ax[2][2])
sns.boxplot(x='class', y='x_std', data=features, ax=ax[3][0])
sns.boxplot(x='class', y='y_std', data=features, ax=ax[3][1])
sns.boxplot(x='class', y='z_std', data=features, ax=ax[3][2])
plt.show()

**part-5**

In [15]:
balanced_dataset = pd.DataFrame()
min_widws = min(features["class"].value_counts())

for Class in classes:
    class_df = features[features["class"]==Class].reset_index(drop=True)
    class_df = class_df.sample(n=min_widws)
    balanced_dataset = balanced_dataset.append(class_df, ignore_index=True)

balanced_dataset = balanced_dataset.sample(frac=1).reset_index()#sample: The frac keyword argument specifies the fraction of rows to return in the random sample, so frac=1 means return all rows (in random order).

In [16]:
print("Class & Data_points")
balanced_dataset["class"].value_counts()

**part-6**

In [17]:

train_df = balanced_dataset.sample(frac=0.6)
test_val = balanced_dataset.drop(train_df.index)
val_df = test_val.sample(frac=0.5)
test_df = test_val.drop(val_df.index)

train_df = train_df.reset_index()
val_df = val_df.reset_index()
test_df = test_df.reset_index()

In [18]:
print("Num of Data Points in Training Data: ", len(train_df))
print("Num of Data Points in Validation Data: ", len(val_df))
print("Num of Data Points in Test Data: ", len(test_df))

**part-7**

**SVM**

In [19]:
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
from sklearn import metrics

In [21]:
train_df.head()

In [24]:
X_train = train_df[['level_0','x_min','y_min','z_min','x_max','z_max','x_avg','y_avg','z_avg','x_std','y_std','z_std']]
X_test=test_df[['level_0','x_min','y_min','z_min','x_max','z_max','x_avg','y_avg','z_avg','x_std','y_std','z_std']]
y_train= train_df[['class']]
y_test= test_df[['class']]
 


In [25]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train,y_train)


In [26]:
clf.predict(X_test)

In [27]:
clf.score(X_test,y_test)

**LR**

In [35]:
from sklearn.linear_model import LogisticRegression
clf1= LogisticRegression(random_state=0).fit(X_train, y_train)
clf1.predict(X_test)

In [36]:
clf1.score(X_test,y_test)

***ANN*

In [41]:
from sklearn.neural_network import MLPClassifier
clf3 = MLPClassifier(solver='lbfgs', alpha=1e-5,
                   hidden_layer_sizes=(5, 2), random_state=1)
clf3.fit(X_train, y_train)

In [42]:
clf3.predict(X_test)

In [43]:
clf3.score(X_test,y_test)